# Introducción a OpenMP y Benchmarking de código paralelo

## Instalación de Dependencias

Vamos a instalar dos SDKs, el compilador Intel OneAPI y el NVHPC Toolkit, en un entorno de Google Colab que utiliza Jupyter Notebook y Debian como sistema base. Aquí está el resumen de los pasos que seguiremos:

### Instalación del SDK de NVHPC

In [2]:
!curl https://developer.download.nvidia.com/hpc-sdk/ubuntu/DEB-GPG-KEY-NVIDIA-HPC-SDK | sudo gpg --dearmor -o /usr/share/keyrings/nvidia-hpcsdk-archive-keyring.gpg
!echo 'deb [signed-by=/usr/share/keyrings/nvidia-hpcsdk-archive-keyring.gpg] https://developer.download.nvidia.com/hpc-sdk/ubuntu/amd64 /' | sudo tee /etc/apt/sources.list.d/nvhpc.list
!sudo apt-get update -y
!sudo apt-get install -y nvhpc

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1626  100  1626    0     0   4947      0 --:--:-- --:--:-- --:--:--  4942
deb [signed-by=/usr/share/keyrings/nvidia-hpcsdk-archive-keyring.gpg] https://developer.download.nvidia.com/hpc-sdk/ubuntu/amd64 /
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/hpc-sdk/ubuntu/amd64  InRelease [2,126 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:7 https://developer.download.nvidia.com/hpc-sdk/ubuntu/amd64  Packages [20.3 kB]
Hit:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease


### Instalación del SDK de Intel OneAPI

In [3]:
!wget -O- https://apt.repos.intel.com/intel-gpg-keys/GPG-PUB-KEY-INTEL-SW-PRODUCTS.PUB | gpg --dearmor | sudo tee /usr/share/keyrings/oneapi-archive-keyring.gpg > /dev/null
!echo "deb [signed-by=/usr/share/keyrings/oneapi-archive-keyring.gpg] https://apt.repos.intel.com/oneapi all main" | sudo tee /etc/apt/sources.list.d/oneAPI.list

!sudo apt-get update -y
!sudo apt-get install -y intel-hpckit

--2023-12-22 15:41:20--  https://apt.repos.intel.com/intel-gpg-keys/GPG-PUB-KEY-INTEL-SW-PRODUCTS.PUB
Resolving apt.repos.intel.com (apt.repos.intel.com)... 23.12.27.65, 2600:1407:3c00:158c::4b23, 2600:1407:3c00:1585::4b23
Connecting to apt.repos.intel.com (apt.repos.intel.com)|23.12.27.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4738 (4.6K) [application/vnd.exstream-package]
Saving to: ‘STDOUT’

-                   100%[===================>]   4.63K  --.-KB/s    in 0s      

2023-12-22 15:41:20 (2.23 GB/s) - written to stdout [4738/4738]

deb [signed-by=/usr/share/keyrings/oneapi-archive-keyring.gpg] https://apt.repos.intel.com/oneapi all main
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Get:2 https://apt.repos.intel.com/oneapi all InRelease [4,455 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/hpc-sdk/ubuntu/amd64  InRelease
Hit:5 

### Instalación del SDK de desarrollo de Ubuntu

In [4]:
!sudo apt-get install -y build-essential make gcc git lshw

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
gcc is already the newest version (4:11.2.0-1ubuntu1).
gcc set to manually installed.
make is already the newest version (4.3-4.1build1).
make set to manually installed.
git is already the newest version (1:2.34.1-1ubuntu1.10).
The following additional packages will be installed:
  pci.ids usb.ids
The following NEW packages will be installed:
  lshw pci.ids usb.ids
0 upgraded, 3 newly installed, 0 to remove and 27 not upgraded.
Need to get 790 kB of archives.
After this operation, 2,988 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 lshw amd64 02.19.git.2021.06.19.996aaad9c7-2build1 [321 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 pci.ids all 0.0~2022.01.22-1 [251 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 usb.ids all 2022.04.02-1 [219 kB]
Fetched 

### Clonado del repositorio y compilación de los ejemplos

In [27]:
!sudo find /opt -name libonnxruntime*

/opt/intel/oneapi/compiler/2024.0/lib/libonnxruntime.1.12.22.721.so
/opt/intel/oneapi/2024.0/lib/libonnxruntime.1.12.22.721.so


In [43]:
sh = """
#!/bin/bash
sudo /opt/nvidia/hpc_sdk/Linux_x86_64/23.11/compilers/bin/makelocalrc -x
. /opt/intel/oneapi/setvars.sh

rm -rf ./QuantumOMP
git clone https://github.com/Neirth/QuantumOMP.git
export LD_LIBRARY_PATH="$LD_LIBRARY_PATH:/opt/intel/oneapi/compiler/2024.0/lib:/opt/intel/oneapi/compiler/2024.0/linux/compiler/lib:/opt/intel/oneapi/compiler/2024.0/linux/compiler/lib/intel64"
export PATH="$PATH:/opt/intel/oneapi/compiler/2024.0/bin:/opt/nvidia/hpc_sdk/Linux_x86_64/23.11/compilers/bin:/usr/local/nvidia/cuda/bin"

cd QuantumOMP && make all
cd build && ./compute_algorithms
"""
with open('script.sh', 'w') as file:
  file.write(sh)

!bash script.sh


 
:: initializing oneAPI environment ...
   script.sh: BASH_VERSION = 5.1.16(1)-release
   args: Using "$@" for setvars.sh arguments: 
:: advisor -- latest
:: ccl -- latest
:: compiler -- latest
:: dal -- latest
:: debugger -- latest
:: dev-utilities -- latest
:: dnnl -- latest
:: dpcpp-ct -- latest
:: dpl -- latest
:: inspector -- latest
:: ipp -- latest
:: ippcp -- latest
:: itac -- latest
:: mkl -- latest
:: mpi -- latest
:: tbb -- latest
:: vtune -- latest
:: oneAPI environment initialized ::
 
Cloning into 'QuantumOMP'...
remote: Enumerating objects: 78, done.
remote: Counting objects: 100% (78/78), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 78 (delta 27), reused 70 (delta 19), pack-reused 0
Receiving objects: 100% (78/78), 32.68 KiB | 1.31 MiB/s, done.
Resolving deltas: 100% (27/27), done.
[ -d build ] || mkdir build
icx -O3 -std=c99 -ldl -Wall -Wextra -Wpedantic -fPIC -Isrc -march=native -ggdb -ipo -qopt-zmm-usage=high -funroll-loops -qopenmp -shared -o

## Ejecución de los benchmarks

## Imprimiendo información del sistema

In [7]:
!sudo lshw

ec8657fe50d0
    description: Computer
    width: 64 bits
    capabilities: smp vsyscall32
  *-core
       description: Motherboard
       physical id: 0
     *-memory
          description: System memory
          physical id: 0
          size: 12GiB
     *-cpu
          product: Intel(R) Xeon(R) CPU @ 2.30GHz
          vendor: Intel Corp.
          physical id: 1
          bus info: cpu@0
          version: 6.63.0
          width: 64 bits
          capabilities: fpu fpu_exception wp vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp x86-64 constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 avx2 smep bmi2 erms invpcid xsaveopt arat md_clear arch_capabilities
          configuration: microcode=4294967295
     *-pci
   